In [ ]:
# Cell 1: Install additional dependencies
!pip install pandas_ta py_vollib

# Cell 2: New imports for options analysis
import pandas_ta as ta
from py_vollib.black_scholes import black_scholes
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega

# Cell 3: Modified Data Ingestion for Options
class OptionsDataIngestionAgent(DataIngestionAgent):
    async def gather_data(self):
        async with aiohttp.ClientSession() as session:
            tasks = [
                self.fetch("https://options-chain-api.com/AAPL", session),
                self.fetch("https://volatility-api.com/historical", session)
            ]
            results = await asyncio.gather(*tasks)
            
        return {
            "options_chain": results[0] or {
                "strikes": [180, 185, 190],
                "expirations": ["2024-06-21", "2024-07-19"],
                "implied_vols": [0.25, 0.28, 0.30]
            },
            "historical": results[1] or pd.Series([185.32, 184.15, 186.75, 183.90])
        }

# Cell 4: Updated Signal Generation with Bollinger/RSI
class OptionsSignalAgent(SignalGenerationAgent):
    def analyze_bands(self, prices):
        df = pd.DataFrame(prices, columns=['close'])
        df.ta.bbands(length=20, std=2, append=True)
        return df[['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']].iloc[-1].to_dict()
    
    def generate_spread_signal(self, data):
        closes = pd.Series(data['historical'])
        bb = self.analyze_bands(closes)
        rsi = ta.rsi(closes, length=14).iloc[-1]
        
        # Bollinger Band strategy logic
        bandwidth = (bb['BBU_20_2.0'] - bb['BBL_20_2.0']) / bb['BBM_20_2.0']
        last_price = closes.iloc[-1]
        
        if bandwidth < 0.1:  # Bollinger Squeeze
            return self._straddle_signal(data, last_price)
        elif last_price > bb['BBU_20_2.0'] and rsi > 70:
            return self._bear_put_spread(data)
        elif last_price < bb['BBL_20_2.0'] and rsi < 30:
            return self._bull_call_spread(data)
        else:
            return self._iron_condor(data)

# Cell 5: Spread Construction Logic
    def _straddle_signal(self, data, price):
        nearest_expiry = data['options_chain']['expirations'][0]
        return {
            "type": "straddle",
            "strike": round(price),
            "expiry": nearest_expiry,
            "legs": [
                {"type": "call", "position": "long"},
                {"type": "put", "position": "long"}
            ],
            "rationale": "Bollinger squeeze + neutral RSI"
        }

    def _bear_put_spread(self, data):
        strikes = sorted(data['options_chain']['strikes'])
        return {
            "type": "put_spread",
            "legs": [
                {"strike": strikes[-1], "position": "long"},
                {"strike": strikes[-2], "position": "short"}
            ],
            "rationale": "Price above upper BB + overbought RSI"
        }

# Cell 6: Risk Assessment for Options
class OptionsRiskSpecialist(RiskAssessmentSpecialist):
    def validate_spread(self, spread, portfolio):
        max_loss = self.calculate_max_loss(spread)
        margin_required = self.calculate_margin(spread)
        
        if max_loss > portfolio['max_loss_per_trade']:
            raise ValueError(f"Max loss {max_loss} exceeds allowed {portfolio['max_loss_per_trade']}")
            
        if margin_required > portfolio['available_margin']:
            raise ValueError("Insufficient margin for spread")
            
        return True

    def calculate_max_loss(self, spread):
        # Implement spread-specific loss calculation
        if spread['type'] == 'straddle':
            return spread['premium_paid']
        elif spread['type'] == 'put_spread':
            return (spread['legs'][0]['strike'] - spread['legs'][1]['strike']) - spread['net_premium']
        return 0

# Cell 7: Hyper-Trading for Options
class OptionsHyperTrading(HyperTradingSpecialist):
    def enhance_spread(self, spread, volatility):
        if volatility > 0.3:
            return self._add_ratio_spread(spread)
        return self._add_hedges(spread)

    def _add_ratio_spread(self, spread):
        # Example: Convert vertical spread to ratio spread
        spread['type'] = "ratio_" + spread['type']
        spread['legs'].append({
            "strike": spread['legs'][-1]['strike'],
            "position": "short",
            "ratio": 2
        })
        return spread

# Cell 8: Greeks Calculation
class OptionsGreekAnalyzer:
    def calculate_risk(self, spread, spot_price, iv, days_to_exp):
        for leg in spread['legs']:
            leg['delta'] = delta(
                'c' if leg['type'] == 'call' else 'p',
                spot_price,
                leg['strike'],
                days_to_exp/365,
                0.05,  # risk-free rate
                iv
            )
            leg['gamma'] = gamma(...)
        spread['portfolio_delta'] = sum(l['delta'] for l in spread['legs'])
        return spread

# Cell 9: Updated Main Pipeline
async def options_pipeline():
    # Stage 1: Options Data
    data_agent = OptionsDataIngestionAgent()
    options_data = await data_agent.gather_data()
    
    # Stage 2/3: Generate Spread Signal
    signal_agent = OptionsSignalAgent()
    spread = signal_agent.generate_spread_signal(options_data)
    
    # Stage 4: Risk Check
    risk_check = OptionsRiskSpecialist()
    if risk_check.validate_spread(spread, {
        'max_loss_per_trade': 1000,
        'available_margin': 50000
    }):
        # Stage 5: Enhance Strategy
        hyper = OptionsHyperTrading()
        enhanced = hyper.enhance_spread(spread, options_data['options_chain']['implied_vols'][0])
        pd.to_pickle(enhanced, "options_trades.pkl")
        return enhanced

# Cell 10: Execute and Display
result = asyncio.run(options_pipeline())
print("Options Spread Strategy:")
print(pd.read_pickle("options_trades.pkl"))